# Agenda

1. Review the iterator protocol
2. Generator functions
    - How to define them
    - How they're different from regular functions
    - Keeping state across invocations
    - How do they work?
3. Generator expressions (aka generator comprehensions)
    - How to define them
    - How to use them

In [1]:
# Lots of objects in Python are iterable

for one_item in 'abcde':
    print(one_item)

a
b
c
d
e


In [2]:
for one_item in [10, 20, 30, 40, 50]:
    print(one_item)

10
20
30
40
50


In [3]:
d = {'a':1, 'b':2, 'c':3}

for one_item in d:
    print(one_item)

a
b
c


In [5]:
# d.items returns an object of type "dict_items"
# it is iterable, also!
# it returns a (key, value) tuple with each iteration

for key, value in d.items():
    print(f'{key}: {value}')

a: 1
b: 2
c: 3


In [6]:
# let's ask d if it is iterable!

i = iter(d)   # normally, don't use "iter" in your programs

In [7]:
i

In [8]:
next(i)

'a'

In [9]:
next(i)

'b'

In [10]:
next(i)

'c'

In [11]:
next(i)

StopIteration: 

In [12]:
def myfunc():
    return 1
    return 2
    return 3


In [13]:
myfunc()

1

In [14]:
import dis  # disassemble our Python code

dis.dis(myfunc)

  2           0 LOAD_CONST               1 (1)
              2 RETURN_VALUE


In [17]:
# here, I define a generator function!
# Python knows it's a generator function because it uses "yield"
# the result of invocing a generator function is a generator object
# generators are iterable -- they know how to behave inside of a "for" loop

def myfunc():
    yield 1
    yield 2
    yield 3

In [18]:
myfunc()

<generator object myfunc at 0x1095f09e0>

In [19]:
myfunc()

<generator object myfunc at 0x1095f0ba0>

In [20]:
myfunc()

<generator object myfunc at 0x1095f0cf0>

In [21]:
g = myfunc()

next(g)  # if g, our generator, is iterable, then it knows how to respond to "next"

1

In [22]:
next(g)

2

In [23]:
next(g)

3

In [24]:
next(g)

StopIteration: 

# What's happening?

Running `next` on a generator object executes the generator's function body through the next `yield`.  You get the value back, and then the generator function goes to sleep just after the `yield`, waking up when you next call `next` on it.

In [25]:
def myfunc():
    print('At start')
    yield 1
    print('In the middle')
    yield 2
    print('Almost done!')
    yield 3
    print('Now I am really done')

In [26]:
g = myfunc()

In [27]:
next(g)

At start


1

In [28]:
next(g)

In the middle


2

In [30]:
next(g)

Almost done!


3

In [31]:
next(g)

Now I am really done


StopIteration: 

In [32]:
def double_numbers(numbers):
    for one_number in numbers:
        yield one_number * 2

In [33]:
double_numbers([10, 20, 30])

<generator object double_numbers at 0x1095ff900>

In [34]:
for one_item in double_numbers([10, 20, 30]):
    print(one_item)

20
40
60


In [37]:
list(double_numbers([10, 20, 30]))

[20, 40, 60]

# Exercise: Only evens

Write a generator function that takes a list (or any other iterable) of integers as an argument. It should return, with each iteration, the next *EVEN* number in that list of integers.  When we get to the end of the input list, then the generator ends.